In [43]:
#Libraries
import pandas as pd
import numpy as np
from collections import OrderedDict #to order the dictionary like the given example
import csv,json

In [3]:
#Use of pandas to visualize data in a friendly format
url='../db/landing/ecommerce/factory14/orders/839012383812.csv'

df=pd.read_csv(url)
df_columns=df.rename(columns={'_job_batch_id': 'daton_user_id', '_job_batch_runtime': 'daton_batch_runtime','_job_user_id':'daton_batch_id'})

df_columns

daton_user_id  daton_batch_runtime  daton_batch_id  \
0            584        1630008352895             980   
1            585        1630008352895             980   

                 admin_graphql_api_id  app_id  \
0  gid://ecommerce/Order/324000907291  580111   
1  gid://ecommerce/Order/323187081243  580111   

                                     billing_address      browser_ip  \
0  [{"first_name":"David","address1":"14 Foxwood ...   108.46.161.60   
1  [{"first_name":"Ramesh","address1":"Level 28, ...  210.10.226.130   

   buyer_accepts_marketing  cancel_reason  cancelled_at  ...  \
0                     True            NaN           NaN  ...   
1                     True            NaN           NaN  ...   

                              token  total_discounts total_line_items_price  \
0  0a2451c04db76d570c07698b12deaeb3              0.0                  499.0   
1  42960f87ca9b8d6ad5b93a17ec8c8abe              0.0                  499.0   

  total_price total_price_usd  total_tax total_weight  \
0       499.0          382.25       0.00            0   
1       499.0          382.25      45.36            0   

                                        transactions           updated_at  \
0  [{"id":3.77738559515E11,"order_id":3.240009072...  2018-04-04T20:36:58   
1  [{"id":3.76813387803E11,"order_id":3.231870812...  2018-04-16T19:42:40   

     user_id  
0  435234134  
1  124312312  

[2 rows x 67 columns]

In [8]:
url_csv='../db/landing/ecommerce/factory14/orders/839012383812.csv'
url_json='../extracts/839012383812_json.json'

KEYS_TO_REPARSE = ["billing_address", "client_details", "customer", 
                    "fulfillments", "line_items", "payment_details", 
                    "shipping_address", "shipping_lines", "transactions"]

result=OrderedDict()
data=[]

#Open CSV file. Only Read access.
with open(url_csv,'r') as csvFile:
    reader = csv.DictReader(csvFile)
    for row in reader:
        
        # Parse the keys that I need to and are not empty
        new_line = {k: json.loads(v) if k in KEYS_TO_REPARSE and v != "" else v for k, v in row.items()}

            #Removes the metadata and saves it in a new index with the names that we need (the cols should always be the same).
            ## I use pop so the data_raw can have data of the rest of the columns, without the metadata
            ##it doesn't matter if the columns change.
        result["daton_user_id"] = new_line.pop('_job_batch_id')
        result["daton_batch_runtime"] = new_line.pop('_job_batch_runtime')
        result["daton_batch_id"] = new_line.pop('_job_user_id')
        result["creation_ts"] = new_line.pop('updated_at')
        result["data_raw"] = new_line

        #Order the dictionary so it can be like the example
        result.move_to_end('data_raw',last=False)
        data.append(result)

#Open TXT file. Write access. It starts appending the dictionaries in the first iteration.
with open(url_json,'w') as jsonFile:
    json.dump(data, jsonFile)



In [55]:
with open('../extracts/839012383812_json.json') as f:
    data = json.loads(f.read())

###Map detailed table
data_raw = [row["data_raw"] for row in data]

#Export the columns of transactions needed
df_transactions =  pd.json_normalize(data_raw,'transactions')
df_ids=df_transactions.filter(['id','order_id'],axis=1).astype(int)

#Export the columns of data_raw needed
df_draw = pd.json_normalize(data_raw)
df_price=df_draw.filter(['user_id','total_price_usd','total_price'],axis=1).astype({'user_id':int,'total_price_usd':float,'total_price':float})

#Export the columns of general json
df = pd.json_normalize(data)
df_creation=df.filter(['creation_ts'],axis=1)
df_creation=pd.to_datetime(df_creation.creation_ts)


df_final=pd.concat([df_ids,df_price,df_creation],axis=1)
df_final.head()


df_final.head()

df_final.to_parquet('../extracts/map_table.gzip',
              compression='gzip') 
pd.read_parquet('../extracts/839012383812_parquet.gzip') 

id      order_id    user_id  total_price_usd  total_price  \
0  376813387803  323187081243  124312312           382.25        499.0   
1  376813387803  323187081243  124312312           382.25        499.0   

          creation_ts  
0 2018-04-16 19:42:40  
1 2018-04-16 19:42:40

In [54]:
###Metrics table
df_transactions =  pd.json_normalize(data_raw,'transactions')
df_orders=df_transactions.filter(['created_at','order_id','amount'],axis=1).astype({'order_id':int,'amount':float})

df_orders['created_at']=pd.to_datetime(df_orders['created_at']).dt.date

df_metrics=df_orders.groupby('created_at', as_index=False).agg({'order_id':'count','amount': 'sum'})

df_metrics.to_parquet('../extracts/metrics.gzip',
              compression='gzip') 
pd.read_parquet('../extracts/metrics.gzip')

created_at  order_id  amount
0  2018-02-05         2   998.0